# Задача ранжирования (machine-learned ranking, MLR)

* [Обучение ранжированию - wiki](https://ru.wikipedia.org/wiki/%D0%9E%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_%D1%80%D0%B0%D0%BD%D0%B6%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8E)
* [Учебник ШАД - Задача ранжирования](https://education.yandex.ru/handbook/ml/article/zadacha-ranzhirovaniya)
* [Устройство поисковых систем: базовый поиск и инвертированный индекс (BM25)](https://habr.com/ru/articles/545634/)

## Что такое ранжирование

Ранжирование — это класс задач машинного обучения с учителем, заключающихся в автоматическом подборе ранжирующей модели по обучающей выборке. Формально задача состоит из следующих составляющих:
* $D$ - коллекции документов
* $Q$ - множества запросов
* $\forall q \in Q D_q \in D$ - набор документов, которые релевантны запросу

Таким образом, задача заключается в том, чтобы для каждого запроса $q \in Q$ отсортировать документы $D_q$ по релевантности.

В этой задаче можно увидеть несколько подзадач:
* Как разметить документы по релевантности к запросу? Как релевантно/не релевантно (классификация)? Как некоторое число (регрессия)?
* Как отсортировать отобранные документы?

Примеры задач ранжирования:
* Web-поиск
* Рекомендательные системы
    * Персонализация новостной ленты
* Поиск синонимов

## Ранжирующие модели

* [Датасет LETOR](https://paperswithcode.com/dataset/istella-letor)
* [Learning to Rank for Information Retrieval](https://www.researchgate.net/publication/220466442_Learning_to_rank_for_information_retrieval_LR4IR_2007)

И документы, и запросы при обучении и инференсе ранжирующей модели представляются как вещественные вектора, состоящие из ранжирующих признаков:
* Значения мер TF, TF-IDF, BM25, и языковой модели соответствия запросу различных зон документа (заголовка, URL, основного текста, текста ссылок);
* Длины и IDF-суммы зон документа;
* Ранги документа, полученные различными вариантами таких алгоритмов ссылочного ранжирования, как PageRank и HITS.

Глобально признаки можно разделить на три группы:
* Признаки, зависящие только от документа. Такие признаки обычно вычисляются на этапе индексирования документов и часто используются для построения показателя статического качества документа, использующегося для повышения эффективности поисковых систем.
    * **PageRank** или длина документа
* Признаки, зависящие только от запроса
* Признаки, зависящие и от документа, и от запроса
    * TF-IDF соответствия документа запросу

Методы для решения задачи обучения ранжированию можно разделить на 3 подхода, в зависимости от используемого входного представления данных и функции штрафа:
1. Поточечный подход (pointwise approach)
    * Предполагается, что каждой паре запрос-документ поставлена в соответствие численная оценка. Задача обучения ранжированию сводится к построению **регрессии**: для каждой отдельной пары запрос-документ необходимо предсказать её оценку.
    * В рамках этого подхода могут применяться многие алгоритмы машинного обучения для задач регрессии. Когда оценки могут принимать лишь несколько значений, также могут использоваться алгоритмы для ординальной регрессии и классификации.
2. Попарный подход (pairwise approach)
    * Построение **бинарного классификатора**, которому на вход поступают два документа, соответствующих одному и тому же запросу, и требуется определить, какой из них лучше.
    * Примеры алгоритмов: RankNet, FRank, RankBoost, RankSVM, IR-SVM.
3. Списочный подход (listwise approach)
    * Построение модели, на вход которой поступают сразу все документы, соответствующие запросу, а на выходе получается их перестановка. Подгонка параметров модели осуществляется для прямой максимизации одной из перечисленных выше метрик ранжирования. Но это часто затруднительно, так как метрики ранжирования обычно не непрерывны и недифференцируемы относительно параметров ранжирующей модели, поэтому прибегают к максимизации неких их приближений или нижних оценок.
    * Примеры алгоритмов: SoftRank, SVMmap, AdaRank, RankGP, ListNet, ListMLE.
    
Модель ранжирования будем обозначать как некоторую функцию от запроса и документа: $a_{\theta}(q, d)$

## Метрики ранжирования

* [Метрики качества ранжирования](https://habr.com/ru/companies/econtenta/articles/303458/)

### Precision@K / Recall@K

Наиболее простые метрики отражают бинарную релевантность запроса и документа: $y(q,d)$ (1 - релевантно, 0 - не релевантно). Но померить бинарную релевантность на всем множестве документов - задача довольно сложная, поэтому будем мерить релевантность **для первых K элементов выдачи**.

$$\text{Precision@K} = \frac{y(q,d) = 1}{K} = \frac{\text{число релевантных элементов}}{K}$$

$$\text{Recall@K} = \frac{y(q,d) = 1}{\min{(K, \sum (y(q,d) = 1)})} = \frac{\text{число релевантных элементов}}{\text{всего релевантных элементов}}$$

### Mean Average Precision (MAP@K)

Average Precision - это метрика, которая равна сумме Precision@K по позициям k только для релевантных элементов:

$$\text{AP@K} = \frac{1}{K} \sum_{k=1}^K \text{Precision@K} y(q,d_q^k)$$

Mean Average Precision - усредненный по коллекции $N$ тестовых запросов Average Precision:

$$\text{MAP@K} = \frac{1}{N} \sum_{j=1}^N AP_j$$

Метрика MAP учитывает порядок выдачи элементов.

### Mean Reciprocal Rank (MRR) - средний обратный ранг

* [Среднеобратный ранг - wiki](https://ru.wikipedia.org/wiki/%D0%A1%D1%80%D0%B5%D0%B4%D0%BD%D0%B5%D0%BE%D0%B1%D1%80%D0%B0%D1%82%D0%BD%D1%8B%D0%B9_%D1%80%D0%B0%D0%BD%D0%B3)

Чем выше в топе запроса релевантный документ, тем лучше работает ранжирование. Будем искать для каждого запроса позицию первого релевантного документа, а затем возьмем от нее обратный ранг (reciproсal rank), и усредним это значение по всем запросам:

$$\text{MRR@K} = \frac{1}{N} \sum_{j=1}^N \frac{1}{\text{rank}}_j$$

MRR@K принимает значения в диапазоне от 0 до 1 и учитывает позицию элементов.

### Normalized discounted cumulative gain (nDCG)

* [Discounted cumulative gain - wiki](https://en.wikipedia.org/wiki/Discounted_cumulative_gain)

Cumulative gain или совокупный прирост — это сумма значений релевантности всех результатов в списке результатов поиска по их грейдам. CG не учитывает ранг (позицию) результата в списке результатов.

CG на определенной позиции ранга $k$ определяется как: $\text{CG@K} = \sum_{k=1}^{K} rel_{k}$

$rel_{k} \in \{0, 1\}$ — оценочная релевантность результата в позиции $k$. 

На значение, вычисленное с помощью функции CG, не влияют изменения порядка результатов поиска. То есть перемещение весьма релевантного документа выше менее релевантного документа с более высоким рейтингом не меняет вычисленное значение для CG.

DCG@K учитывает порядок элементов в списке. Идея DCG@K заключается в том, что высокорелевантные документы, находящиеся ниже в списке результатов поиска, должны подвергаться штрафам:

$$\text{DCG@K} = \sum_{k=1}^{K} {\frac{rel_{k}}{\log_{2}(k+1)}}$$

Сравнение производительности поисковой системы от одного запроса к другому не может быть последовательно достигнуто с использованием только DCG, поэтому совокупный прирост на каждой позиции для выбранного значения должен быть нормализован по всем запросам. Это делается путем сортировки всех соответствующих документов в корпусе по их относительной релевантности. Для запроса nDCG рассчитывается как: 

$$\text{nDCG@K} = \frac{\text{DCG@K}}{\text{IDCG@K}}$$

где $\text{IDCG@K} = \sum_{k=1}^K \frac{1}{\log_{2}(k+1)}$ - Ideal DCG@K - максимальное значение DCG@K

Значения nDCG для всех запросов можно усреднить, чтобы получить оценку средней производительности алгоритма ранжирования поисковой системы.

Основная трудность, возникающая при использовании nDCG, заключается в отсутствии идеального порядка результатов, когда доступна только частичная обратная связь по релевантности.

nDCG@K принимает значения в диапазоне от 0 до 1.

### Метрики на основе ранговой корреляции

* Ранговый коэффициент корреляции Кендэлла
* Ранговый коэффициент корреляции Спирмена

### Метрики на основе каскадной модели поведения

* Expected Reciprocal Rank
* pFound